In [ ]:
import traitlets as T
import ipywidgets as W
from ipyradiant import MultiPanelSelect, PathLoader, FileManager
from rdflib import Graph, Namespace, URIRef
from rdflib.namespace import NamespaceManager

## Start with creating a class composing of a checkbox, and the multipanel select.
### We will also create a CustomURIRef class in order to get a 'pretty' representation of a URIRef while maintaining access to the full URI.

In [ ]:
class CustomURIRef:
    # TODO: investigate using rdflib.URIRef as a baseclass
    def __init__(self, uri, namespace_manager):
        self.uri = URIRef(uri)
        self.namespace_manager = namespace_manager

    
    def __repr__(self):
        return URIRef(self.uri).n3(self.namespace_manager)

In [ ]:
class objectLiteralApp(W.VBox):
    graph = T.Instance(Graph, allow_none=True)
    lit_button = T.Instance(W.Button)
    multiselect = T.Instance(MultiPanelSelect)
    namespaces = T.Dict(default_value={}, allow_none = True)



    def __init__(self, graph: Graph = None, *args, **kwargs):
        if graph is not None:
            kwargs["graph"] = graph
        if "layout" not in kwargs:
            kwargs["layout"] = {
                "min_height": "400px",
                "flex": "1",
            }
        super().__init__(*args, **kwargs)
        
        for key,value in self.namespaces.items():
            self.graph.namespace_manager.bind(key, value)
        
        self.lit_button.on_click(self.populate_predicates)
        self.children = [self.multiselect, self.lit_button]

    @T.default("graph")
    def make_default_graph(self):
        return Graph()

    @T.default("lit_button")
    def make_default_button(self):
        
        layout = W.Layout(width='300px', height='40px') #set width and height

        button = W.Button(
            description='Add predicates where object is a literal',
            disabled=False,
            layout = layout
        )    

        return button

    @T.default("multiselect")
    def make_default_multiselect(self):
        return MultiPanelSelect(data=[], left_panel_text="Available Predicates", right_panel_text="Predicates to Collapse")
    

    def populate_predicates(self, b):
        # query for all predicates where object is not literal
        predicates_to_uris = """
            SELECT DISTINCT  ?p 
            WHERE {
                    ?s ?p ?o .
                FILTER (!isLiteral(?o))
                } 
            """
        q1 = self.graph.query(predicates_to_uris)
        q1_results = set([result[0] for result in q1])

        # query for predicates where object IS literal
        predicates_to_literals = """
            SELECT DISTINCT  ?p 
            WHERE {
                    ?s ?p ?o .
                FILTER isLiteral(?o)
                } 
            """
        q2 = self.graph.query(predicates_to_literals)
        q2_results = set([result[0] for result in q2])
        #take difference of sets and turn to list
        set_diff = list(q2_results-q1_results)
        
        
        truncated_uris = []
        for ref in set_diff:
            truncated_uris.append(CustomURIRef(uri = ref, namespace_manager = self.graph.namespace_manager))
        self.multiselect.selected_things_list = truncated_uris
            

We can borrow the loading widgets seen in other examples like the Tab App in order to get a graph to play around with.

In [ ]:
lw = FileManager(loader=PathLoader(path="data"))
lw

In [ ]:
graph = lw.graph
ola = objectLiteralApp(graph=graph, namespaces={URIRef('http://www.w3.org/'):'w3'})
ola

Now lets say we want to access one of the predicates. We can do this while retaining full access to the URI as follows:

In [ ]:
first_pred = ola.multiselect.selected_things_list[0]
first_pred.uri